In [1]:
import os
os.chdir('../')

### from "sample noise data pair by Euler and save pt.ipynb"

In [2]:
import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_120.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

torch.Size([1024, 3, 64, 64]) torch.Size([1024, 3, 64, 64])


In [3]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False

###############################################################################
# 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
###############################################################################
sys.argv = [
    "main.py",
    "--config", "imagenet64.yml",  # 사용하려는 config
    "--sample",
    "--sample_type", "generalrbfsolver",
    "--dpm_solver_type", "data_prediction",
    "--dpm_solver_order", "3",
    "--timesteps", "5",
    "--skip_type", "logSNR",
    "--kernel_name", "Gaussian",
    "--subintervals", "100",
    "--log_scale_max", "6.0",
    "--scale_dir", "/data/data/general_rbf_scales",
    "--ni"
]

###############################################################################
# 2) 인자/설정 로드
###############################################################################
args, config = parse_args_and_config()

###############################################################################
# 3) Diffusion 객체 생성 -> 모델 로딩
###############################################################################
diffusion = Diffusion(args, config, rank=0)
diffusion.prepare_model()
diffusion.model.eval()

###############################################################################
# 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
###############################################################################
device = diffusion.device


INFO - main.py - 2025-03-24 13:21:07,452 - Using device: cuda


[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


In [4]:
n = 256
noise_batch = noise[:n].to(device)
target_batch = target[:n].to(device)
(pred, optimal_gammas), _ = diffusion.sample_image(noise_batch, diffusion.model, target=target_batch)
loss = F.mse_loss(pred, target_batch)
print('loss :', loss)


/data/data/general_rbf_scales/NFE=5,p=3,kernel=Gaussian.npy  saved!
loss : tensor(0.0329, device='cuda:0')


### Batch Mode

In [4]:
import sys
import torch
import numpy as np
from main import parse_args_and_config, Diffusion
from tqdm import tqdm

import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_120.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

model = None
for NFE in tqdm([5, 6, 8, 10, 12, 15, 20, 25]):
    for kernel_name in ['Gaussian']:
        sys.argv = [
            "main.py",
            "--config", "imagenet64.yml",  # 사용하려는 config
            "--sample",
            "--sample_type", "generalrbfsolver",
            "--dpm_solver_type", "data_prediction",
            "--dpm_solver_order", "3",
            "--timesteps", str(NFE),
            "--skip_type", "logSNR",
            "--kernel_name", str(kernel_name),
            "--subintervals", "100",
            "--log_scale_max", "6.0",
            "--scale_dir", "/data/data/general_rbf_scales",
            "--verbose", "critical",
            "--ni",
        ]

        args, config = parse_args_and_config()
        diffusion = Diffusion(args, config, rank=0)

        if model is None:
            diffusion.prepare_model()
            diffusion.model.eval()
            model = diffusion.model
        device = diffusion.device

        n = 256
        noise_batch = noise[:n].to(device)
        target_batch = target[:n].to(device)
        (pred, _), _ = diffusion.sample_image(noise_batch, model, target=target_batch)
        print(F.mse_loss(pred, target_batch))


torch.Size([1024, 3, 64, 64]) torch.Size([1024, 3, 64, 64])


  0%|          | 0/8 [00:00<?, ?it/s]

[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


 12%|█▎        | 1/8 [00:03<00:25,  3.61s/it]

/data/data/general_rbf_scales/NFE=5,p=3,kernel=Gaussian.npy  saved!
tensor(0.0329, device='cuda:0')


...

/data/data/general_rbf_scales/NFE=15,p=3,kernel=Gaussian.npy  saved!
tensor(0.0067, device='cuda:0')


 88%|████████▊ | 7/8 [00:48<00:08,  8.98s/it]

/data/data/general_rbf_scales/NFE=20,p=3,kernel=Gaussian.npy  saved!
tensor(0.0069, device='cuda:0')


100%|██████████| 8/8 [01:04<00:00,  8.04s/it]

/data/data/general_rbf_scales/NFE=25,p=3,kernel=Gaussian.npy  saved!
tensor(0.0089, device='cuda:0')
